In [3]:
import numpy as np
#import scipy as sc
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import tensorflow as tf

In [5]:
import imp
en = imp.load_source('encode', '/home/arturas/Projects/ProteoDNN/Utils/encode.py')

In [6]:
oxy = pd.read_csv('data/OxidationAll.txt', sep=',')

In [7]:
oxy.dtypes

Peptide                        object
Proteins                       object
Identification Score          float64
Pair Score                    float64
[L]ID                           int64
[L]Modifications              float64
[L]Mass                       float64
[L]Charge                       int64
[L]m/z                        float64
[L]Retention Time (min)       float64
[H]ID                           int64
[H]Modifications               object
[H]Mass                       float64
[H]Charge                       int64
[H]m/z                        float64
[H]Retention Time (min)       float64
Mass Difference (Observed)    float64
Mass Difference (Expected)    float64
Mass Difference (Delta)       float64
Retention (Delta)             float64
Unnamed: 20                   float64
dtype: object

In [8]:
oxy_ = oxy[(oxy['[L]Retention Time (min)'] > 20) & (oxy['[L]Retention Time (min)'] < 80)]

In [9]:
peptides = [np.array(en.encode_aa_counts(pep)).reshape(-1,1) for pep in oxy_.iloc[:,0]]
peptides = np.concatenate(peptides,axis=1)
peptides = peptides.T

In [10]:
ret_delta = oxy_.iloc[:,19]

In [12]:
#normalize features
peptides_norm = (peptides - np.mean(peptides, axis=0))/np.std(peptides,axis=0)
peptides_norm.shape

(178884, 20)

In [50]:
def append_bias_reshape(features,labels):
    n_training_samples = features.shape[0]
    n_dim = features.shape[1]
    f = np.reshape(np.c_[np.ones(n_training_samples),features],[n_training_samples,n_dim + 1])
    l = np.reshape(labels,[n_training_samples,1])
    return f, l

#Xdat, Ydat = append_bias_reshape(peptides_norm, ret_delta)
Xdat = peptides#_norm
Ydat = ret_delta

In [51]:
n_dim = Xdat.shape[1]

rnd_indices = np.random.rand(len(Xdat)) < 0.80

trX = Xdat[rnd_indices]
trY = Ydat[rnd_indices].reshape(-1,1)
teX = Xdat[~rnd_indices]
teY = Ydat[~rnd_indices].reshape(-1,1)

/home/arturas/anaconda3/envs/tensorflow/lib/python3.5/site-packages/ipykernel/__main__.py:6: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
/home/arturas/anaconda3/envs/tensorflow/lib/python3.5/site-packages/ipykernel/__main__.py:8: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


In [66]:
trY[1:5]

array([[  7.39 ],
       [  7.79 ],
       [ 13.45 ],
       [  9.315]])

In [53]:
# Weight init and model declaration
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))

def model(X, w_h, w_h2, w_o ):
    h = tf.nn.relu(tf.matmul(X, w_h))
    h2 = tf.nn.relu(tf.matmul(h, w_h2))
    #h3 = tf.nn.relu(tf.matmul(h2, w_h3))
    #h4 = tf.nn.relu(tf.matmul(h3, w_h4))
    return tf.matmul(h2, w_o)

In [64]:
# Accuracy measure
def accMeas(mat, trueMat):
    mat[mat>0.5] = 1.0
    mat[mat<=0.5] = 0.0
    correct = np.zeros(mat.shape)
    correct[mat==trueMat] = 1
    miscrate =  correct.mean(axis=0)
    return(miscrate)

def MSE(pred, trueDat):
    return np.sum((pred-trueDat)**2) / len(pred)

In [67]:
# Creating symbolic variables and initializing weights

X = tf.placeholder("float", [None, 20])
Y = tf.placeholder("float", [None, 1])

w_h = init_weights([20, 100])
w_h2 = init_weights([100,50])
#w_h3 = init_weights([350,150])
w_o = init_weights([50, 1])

py_x = model(X, w_h, w_h2, w_o)

In [68]:
# Cost function and optimizer
#cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(py_x, Y))
cost = tf.reduce_mean(tf.square(py_x-Y))
#train_op = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)
train_op = tf.train.GradientDescentOptimizer(0.03).minimize(cost)
#predict_op = tf.nn.sigmoid(py_x)
predict_op = py_x

In [70]:
# Launch the graph in a session
with tf.Session() as sess:
    # you need to initialize all variables
    tf.initialize_all_variables().run()

    for i in range(1000):
        for start, end in zip(range(0, len(trX), 200), range(200, len(trX), 200)):
            sess.run(train_op, feed_dict={X: trX[start:end], Y: trY[start:end]})
        if(i%100==0):
            print(i, (np.abs(sess.run(predict_op, feed_dict={X: teX, Y: teY[:]}) - teY[:])).mean(axis=0))
    print(i, (np.abs(sess.run(predict_op, feed_dict={X: teX, Y: teY[:]}) - teY[:])).mean(axis=0))
    
    print("Training Error - correct: = {}".format(MSE(sess.run(predict_op, feed_dict={X: trX, Y: trY}),trY)))
    print("Test Error - correct:  = {}".format(MSE(sess.run(predict_op, feed_dict={X: teX, Y: teY}),teY)))
    #print("New data:")
    #print(np.round(sess.run(predict_op, feed_dict={X: testExt[1:5],Y:teY[1:5]})))
    #np.savetxt("/home/arturas/Desktop/pred.txt",sess.run(predict_op, feed_dict={X: trX, Y: trY}), delimiter='\t')

0 [ 8.56751451]
100 [ 8.56751305]
200 [ 8.56751305]
300 [ 8.56751305]
400 [ 8.56751305]
500 [ 8.56751305]
600 [ 8.56751305]
700 [ 8.56751306]
800 [ 8.56751306]
900 [ 8.56751306]
999 [ 8.56751306]
Training Error - correct: = 90.69949208782508
Test Error - correct:  = 89.0102760767894
